In [1]:
import torch
import gurobipy as gp

In [2]:
m = gp.Model()
v1 = m.addVar(vtype=gp.GRB.BINARY)
v2 = m.addVar(vtype=gp.GRB.BINARY)
m.addConstr(v1 >= v2)
m.setObjective(v1 + v2, sense=gp.GRB.MAXIMIZE)
m.optimize()

Restricted license - for non-production use only - expires 2026-11-23
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[x86] - Darwin 22.4.0 22E252)

CPU model: Intel(R) Core(TM) i9-9980HK CPU @ 2.40GHz
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 1 rows, 2 columns and 2 nonzeros
Model fingerprint: 0x068e17f7
Variable types: 0 continuous, 2 integer (2 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [0e+00, 0e+00]
Found heuristic solution: objective 2.0000000

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 16 available processors)

Solution count 1: 2 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.000000000000e+00, best bound 2.000000000000e+00, gap 0.0000%


In [3]:
from learn.info import ModelInfo

In [4]:
info = ModelInfo.from_model(m)

In [5]:
info

[(0.0, 1.0, 'B'), (0.0, 1.0, 'B')]
[([0, 1], [1.0, -1.0], '>=', 0.0)]
[{0: 1.0, 1: 1.0}, -1]

In [6]:
from learn.feature import VarFeature, ConFeature, EdgFeature

In [7]:
from learn.train import Inst

In [8]:
inst = Inst(
    [VarFeature.from_info(info.var_info, info.obj_info)],
    [ConFeature.from_info(info.con_info)],
    [EdgFeature.from_info(info.con_info)],
    [[v.x for v in m.getVars()]]
)

In [9]:
import torch
from learn.model import FocalLoss, SpGAT
import torch.optim as optim

model = SpGAT(
    nfeat=inst.xs[2][0].shape[1],
    nhid=64,
    nclass=2,
    dropout=0.5,
    nheads=6,
    alpha=0.2
)

optimizer = optim.Adam(model.parameters(), lr=1e-2, weight_decay=5e-3)

In [10]:
c_v_edges, v_c_edges, node_features, edge_features = inst.xs
ys = inst.ys

In [11]:
import random

for epoch in range(10):
    
    model.train()
    optimizer.zero_grad()
    
    agg_loss = 0
    for i in range(5):
        inst_idx = random.randint(0, len(ys) - 1)
        train_idx = torch.as_tensor(range(2), dtype=torch.int32)
        
        output, edge_features[inst_idx] = model(
            node_features[inst_idx], 
            c_v_edges[inst_idx], 
            v_c_edges[inst_idx], 
            edge_features[inst_idx].detach()
        )
        fl = FocalLoss()
        loss = fl(output[train_idx], ys[inst_idx][train_idx])
        agg_loss += loss
        
    print(output[train_idx].detach().numpy(), ys[inst_idx][train_idx].detach().numpy())
    print(agg_loss)
    
    agg_loss.backward()
    optimizer.step()

[[0.5488219 0.451178 ]
 [0.5580024 0.4419976]] [1 1]
tensor(1.0384, grad_fn=<AddBackward0>)
[[0.00244018 0.9975598 ]
 [0.00242765 0.99757236]] [1 1]
tensor(0.1146, grad_fn=<AddBackward0>)
[[1.0370454e-10 1.0000000e+00]
 [1.0645256e-10 1.0000000e+00]] [1 1]
tensor(0.1133, grad_fn=<AddBackward0>)
[[3.8930806e-20 1.0000000e+00]
 [4.0239603e-20 1.0000000e+00]] [1 1]
tensor(0.1133, grad_fn=<AddBackward0>)
[[3.9185967e-31 1.0000000e+00]
 [3.9104731e-31 1.0000000e+00]] [1 1]
tensor(0.1133, grad_fn=<AddBackward0>)
[[5.80e-43 1.00e+00]
 [5.58e-43 1.00e+00]] [1 1]
tensor(0.1133, grad_fn=<AddBackward0>)
[[0. 1.]
 [0. 1.]] [1 1]
tensor(0.1133, grad_fn=<AddBackward0>)
[[0. 1.]
 [0. 1.]] [1 1]
tensor(0.1133, grad_fn=<AddBackward0>)
[[0. 1.]
 [0. 1.]] [1 1]
tensor(0.1133, grad_fn=<AddBackward0>)
[[0. 1.]
 [0. 1.]] [1 1]
tensor(0.1133, grad_fn=<AddBackward0>)


In [12]:
output

tensor([[0.0000e+00, 1.0000e+00],
        [0.0000e+00, 1.0000e+00],
        [3.9067e-38, 1.0000e+00]], grad_fn=<SoftmaxBackward0>)

In [13]:
train_idx

tensor([0, 1], dtype=torch.int32)

In [14]:
output

tensor([[0.0000e+00, 1.0000e+00],
        [0.0000e+00, 1.0000e+00],
        [3.9067e-38, 1.0000e+00]], grad_fn=<SoftmaxBackward0>)

In [15]:
assert 1 == 2

AssertionError: 

In [ ]:
def train(epoch, num):
    global data_edge_features
    t = time.time()

    output, data_edge_features[num] = model(data_features[num], data_edge_A[num], data_edge_B[num], data_edge_features[num].detach())
    print(data_solution[num][idx_train])

    lf = Focal_Loss(torch.as_tensor(data_labels[num]))
    loss_train = lf(output[idx_train], data_solution[num][idx_train])

    return loss_train

In [ ]:
t_total = time.time()
loss_values = []
for epoch in range(args.epochs):
    model.train()
    optimizer.zero_grad()
    now_loss = 0
    
    for i in range(5):
        now_data = random.randint(0, data_num - 1)
        now_loss += train(epoch, now_data)
        
    loss_values.append(now_loss)
    now_loss.backward()
    optimizer.step()
    print('Epoch: {:04d}'.format(epoch+1),
          'loss_train: {:.4f}'.format(now_loss))

    torch.save(model.state_dict(), '{}.pkl'.format(epoch))
    if loss_values[-1] < best:
        best = loss_values[-1]
        best_epoch = epoch
        bad_counter = 0
    else:
        bad_counter += 1

In [ ]:
# Model and optimizer
model = SpGAT(nfeat=data_features[0].shape[1],    # Feature dimension
            nhid=args.hidden,             # Feature dimension of each hidden layer
            nclass=int(data_solution[0].max()) + 1, # Number of classes
            dropout=args.dropout,         # Dropout
            nheads=args.nb_heads,         # Number of heads
            alpha=args.alpha)             # LeakyReLU alpha coefficient

optimizer = optim.Adam(model.parameters(),    
                       lr=args.lr,                        # Learning rate
                       weight_decay=args.weight_decay)    # Weight decay to prevent overfitting

if args.cuda: # Move to GPU
    model.to(device)
    for now_data in range(data_num):
        data_features[now_data] = data_features[now_data].to(device)
        data_labels[now_data] = data_labels[now_data].to(device)
        data_solution[now_data] = data_solution[now_data].to(device)
        data_edge_A[now_data] = data_edge_A[now_data].to(device)
        data_edge_B[now_data] = data_edge_B[now_data].to(device)
        data_edge_features[now_data] = data_edge_features[now_data].to(device)
        data_idx_train[now_data] = data_idx_train[now_data].to(device)


for now_data in range(data_num):
    data_features[now_data] = Variable(data_features[now_data])
    data_edge_A[now_data] = Variable(data_edge_A[now_data])
    data_edge_B[now_data] = Variable(data_edge_B[now_data])
    data_solution[now_data] = Variable(data_solution[now_data])
    # Define computation graph for automatic differentiation

def train(epoch, num):
    global data_edge_features
    t = time.time()

    output, data_edge_features[num] = model(data_features[num], data_edge_A[num], data_edge_B[num], data_edge_features[num].detach())
    print(data_solution[num][idx_train])

    lf = Focal_Loss(torch.as_tensor(data_labels[num]))
    loss_train = lf(output[idx_train], data_solution[num][idx_train])

    return loss_train

t_total = time.time()
loss_values = []
bad_counter = 0
best = args.epochs + 1
best_epoch = 0
for epoch in range(args.epochs):
    model.train()
    optimizer.zero_grad()
    now_loss = 0
    for i in range(5):
        now_data = random.randint(0, data_num - 1)
        now_loss += train(epoch, now_data)
    loss_values.append(now_loss)
    now_loss.backward()
    optimizer.step()
    print('Epoch: {:04d}'.format(epoch+1),
          'loss_train: {:.4f}'.format(now_loss))

    torch.save(model.state_dict(), '{}.pkl'.format(epoch))
    if loss_values[-1] < best:
        best = loss_values[-1]
        best_epoch = epoch
        bad_counter = 0
    else:
        bad_counter += 1